# 問題38: 単語の出現頻度のヒストグラム

単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，縦軸は出現頻度をとる単語の種類数を表す．

In [ ]:
# 問題30で実装した関数を読み込む
import sys
sys.path.append('../..')  # 親ディレクトリをパスに追加
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 日本語表示のための設定
plt.rcParams['font.family'] = 'IPAexGothic'

# 形態素解析結果を読み込む関数
def load_mecab_result(file_path):
    """
    MeCabの解析結果ファイルを読み込み、各形態素を辞書のリストとして返す関数
    
    Args:
        file_path (str): MeCab出力ファイルのパス
        
    Returns:
        list: 文のリスト。各文は形態素（辞書）のリスト
    """
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # EOSは文の区切り
            if line == 'EOS\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
                
            # 空行をスキップ
            if line == '\n':
                continue
                
            # タブで分割して表層形とそれ以外の情報に分ける
            try:
                surface, info = line.split('\t')
                
                # カンマで分割して品詞情報などを取得
                info_items = info.split(',')
                
                # 形態素情報を辞書として格納
                morpheme = {
                    'surface': surface,
                    'base': info_items[6],
                    'pos': info_items[0],
                    'pos1': info_items[1]
                }
                
                current_sentence.append(morpheme)
            except:
                # 不正な形式の行をスキップ
                continue
    
    # 最後の文が追加されていない場合に追加
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

# 単語の出現頻度を計算する関数
def count_word_frequency(sentences):
    """
    形態素解析結果から単語の出現頻度を計算する関数
    
    Args:
        sentences (list): 文のリスト。各文は形態素（辞書）のリスト
        
    Returns:
        Counter: 単語の出現頻度を格納したCounterオブジェクト
    """
    # 全ての形態素の表層形を抽出
    words = [morpheme['surface'] for sentence in sentences for morpheme in sentence]
    
    # 単語の出現回数をカウント
    word_counts = Counter(words)
    
    return word_counts

# 出現頻度のヒストグラムを描画する関数
def plot_frequency_histogram(word_counts, max_freq=100, bins=30):
    """
    単語の出現頻度のヒストグラムを描画する関数
    
    Args:
        word_counts (Counter): 単語の出現頻度を格納したCounterオブジェクト
        max_freq (int): ヒストグラムに含める最大頻度
        bins (int): ヒストグラムのビン（区間）の数
    """
    # 出現頻度のリストを作成
    frequencies = list(word_counts.values())
    
    # 頻度の上限を設定（グラフを見やすくするため）
    frequencies = [freq for freq in frequencies if freq <= max_freq]
    
    # ヒストグラムの描画
    plt.figure(figsize=(12, 8), dpi=300)
    
    # ヒストグラムを描画
    n, bins, patches = plt.hist(frequencies, bins=bins, color='skyblue', edgecolor='black', alpha=0.7)
    
    # グラフの装飾
    plt.title('単語の出現頻度のヒストグラム', fontsize=16)
    plt.xlabel('出現頻度', fontsize=14)
    plt.ylabel('単語の種類数', fontsize=14)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # 対数スケールを使用（オプション）
    # plt.yscale('log')
    
    plt.tight_layout()
    
    return plt.gcf()

In [ ]:
# 形態素解析結果の読み込み
file_path = '../../data/neko.txt.mecab'
sentences = load_mecab_result(file_path)

# 単語の出現頻度を計算
word_counts = count_word_frequency(sentences)

# 出現頻度の基本統計量を表示
frequencies = list(word_counts.values())
print(f"単語の種類数: {len(frequencies)}")
print(f"最小出現頻度: {min(frequencies)}")
print(f"最大出現頻度: {max(frequencies)}")
print(f"平均出現頻度: {np.mean(frequencies):.2f}")
print(f"中央値出現頻度: {np.median(frequencies):.2f}")

# 出現頻度の分布を表示
freq_of_freq = Counter(frequencies)
print("\n出現頻度ごとの単語数:")
for freq, count in sorted(freq_of_freq.items())[:10]:
    print(f"出現頻度 {freq} の単語数: {count}")

# 出現頻度のヒストグラムを描画（最大頻度を100に制限）
fig = plot_frequency_histogram(word_counts, max_freq=100, bins=30)
plt.show()

# 出現頻度のヒストグラムを描画（低頻度域を詳細に表示）
fig2 = plot_frequency_histogram(word_counts, max_freq=20, bins=20)
plt.title('単語の出現頻度のヒストグラム（低頻度域）', fontsize=16)
plt.show()

# 出現頻度のヒストグラムを描画（対数スケール）
plt.figure(figsize=(12, 8), dpi=300)
plt.hist(frequencies, bins=100, color='skyblue', edgecolor='black', alpha=0.7)
plt.title('単語の出現頻度のヒストグラム（対数スケール）', fontsize=16)
plt.xlabel('出現頻度', fontsize=14)
plt.ylabel('単語の種類数', fontsize=14)
plt.yscale('log')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 解説

この問題では、単語の出現頻度のヒストグラムを描画しました。ヒストグラムは、横軸に出現頻度、縦軸に各出現頻度をとる単語の種類数を表示しています。

### 実装のポイント

1. **頻度の計算**: 問題35で実装した関数を使用して、各単語の出現頻度を計算しています。

2. **ヒストグラムの作成**: matplotlibの`hist`関数を使用して、出現頻度のヒストグラムを作成しています。

3. **表示範囲の調整**: 出現頻度の分布は非常に偏っているため、最大頻度を制限して見やすくしています。

4. **ビン数の設定**: ヒストグラムのビン（区間）の数を適切に設定して、分布の特徴が見やすくなるようにしています。

5. **対数スケール**: 出現頻度の分布は非常に偏っているため、縦軸を対数スケールにすることで、低頻度の単語の分布も見やすくなります。

### 出現頻度分布の特徴

単語の出現頻度分布には、一般的に以下のような特徴があります：

1. **ロングテール分布**: 少数の単語が非常に高い頻度で出現し、大多数の単語は低い頻度でしか出現しません。これは「ロングテール分布」と呼ばれる特徴的な分布です。

2. **Zipfの法則**: 単語の出現頻度はその順位に反比例する傾向があります（問題39で詳しく扱います）。

3. **低頻度語の多さ**: 多くの単語は1回か2回しか出現しません。これらの「ハプックスレゴメナ（hapax legomena、1回だけ出現する単語）」は、語彙の多様性を示す指標となります。

### 結果の考察

ヒストグラムから、以下のような特徴が観察できます：

1. **頻度1の単語が最も多い**: 1回だけ出現する単語が最も多く、出現頻度が増えるにつれて、その頻度をとる単語の数は急激に減少します。

2. **高頻度語の少なさ**: 高頻度（例えば50回以上）で出現する単語は非常に少なく、これらは主に助詞や助動詞などの機能語です。

3. **分布の形状**: 出現頻度の分布は、典型的なZipf分布の特徴を示しています。これは自然言語の普遍的な特性の一つです。

このような出現頻度の分布は、テキストの語彙の豊かさや多様性を示す指標となります。また、自然言語処理のタスクにおいて、低頻度語の扱いは重要な課題の一つです。